In [1]:
from glob import glob

CORPUS_PATH = "livedoor_data/text/sports-watch/*.txt"
text_corpus = glob(CORPUS_PATH)
start_token = "<"
end_token = ">"
entire_text = ""

for filepath in text_corpus:
    if filepath == "livedoor_data/text/sports-watch/LICENSE.txt":
        continue
    else:
        tmp_text = start_token
        with open(filepath, "r", encoding="utf-8") as f:
            #skip first 2 rows for each document
            #1st row:referred URL, 2nd row:the article-written date
            for i in range(2):
                next(f)
            tmp_text += f.read()
            entire_text += tmp_text
            entire_text += end_token
        
len(entire_text)

622094

In [2]:
import sentencepiece as spm

tokenizer = spm.SentencePieceProcessor()
tokenizer.Load("wiki_data/wikiextractor/spm.model")

str_tokens = tokenizer.EncodeAsPieces(entire_text)[1:]
print(str_tokens[:100])
print("length of str_tokens:", len(str_tokens))

int_tokens = []
for token in str_tokens:
    int_tokens.append(tokenizer.piece_to_id(token))
    
print(int_tokens[:100])
print("length of int_tokens:", len(int_tokens))

['<', '【', 'S', 'p', 'or', 't', 's', '▁W', 'at', 'ch', '】', '秋', '山', '成', '勲', '、', 'メール', 'で', '吉田', 'に', '対戦', '迫', 'った', '!', '?', '▁', '今', '月', '8', '日', '、', '都', '内', 'ホテル', 'では', '、', '総合', '格闘', '家', '・', '吉田', '秀', '彦', 'の', '引退', '試合', '興行', '「', 'A', 'ST', 'RA', '」', 'の', '開催', 'が', '発表された', '。', '▁', 'バル', 'セ', 'ロ', 'ナ', '五', '輪', '柔道', '金', 'メ', 'ダ', 'リスト', 'としての', '実', '績', 'を', '引', 'っ', 'さ', 'げ', '、', '2002', '年に', 'プロ', '総合', '格闘', '家', 'に転', '向', '。', '以後', '、', '数', '々', 'の', '死', '闘', 'を', '繰り', '広', 'げ', 'てきた', '吉田']
length of str_tokens: 418833
[2003, 6050, 130, 334, 531, 268, 160, 1677, 1055, 945, 6051, 1341, 63, 391, 5563, 3, 4893, 14, 4366, 10, 2789, 2840, 105, 947, 2607, 6, 702, 19, 62, 30, 3, 949, 155, 2839, 36, 3, 1498, 5311, 122, 11, 4366, 2046, 2299, 4, 1583, 569, 4972, 20, 141, 1997, 2774, 18, 4, 814, 9, 2508, 5, 6, 1077, 266, 83, 90, 810, 1216, 5470, 220, 188, 246, 1610, 1764, 291, 4879, 8, 1301, 360, 202, 862, 3, 1386, 72, 524, 1498, 5311, 122, 3469, 

In [3]:
import numpy as np
from keras.utils.np_utils import to_categorical

input_texts, target_texts = [], []
seq_length = 20
num_vocabs = 8000

for i in range(0, len(int_tokens) - seq_length, 1):
    input_texts.append(int_tokens[i: i + seq_length])
    target_texts.append(int_tokens[i + seq_length])
    
target_texts_one_hot = to_categorical(target_texts, num_classes=num_vocabs)
X = np.array(input_texts)
y = np.array(target_texts_one_hot)
X.shape, y.shape

Using TensorFlow backend.


((418813, 20), (418813, 8000))

In [4]:
from keras.models import Model
from keras.layers import Input, Embedding, CuDNNLSTM, Dropout, Dense

hidden_dims = 256
embed_dims = 300

input_text = Input((None,))
x = Embedding(num_vocabs, embed_dims)(input_text)
x = CuDNNLSTM(hidden_dims)(x)
#x = Dropout(0.2)(x)
output_text = Dense(num_vocabs, activation="softmax")(x)

model = Model(input_text, output_text)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 300)         2400000   
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 256)               571392    
_________________________________________________________________
dense_1 (Dense)              (None, 8000)              2056000   
Total params: 5,027,392
Trainable params: 5,027,392
Non-trainable params: 0
_________________________________________________________________


In [5]:
from keras.optimizers import Adam

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=["acc"])

In [6]:
epochs = 50
batch_size = 128

model.fit(X, y, epochs=epochs, batch_size=batch_size)

Epoch 1/50
418813/418813 [==============================] - 32s 76us/step - loss: 5.9349 - acc: 0.1178
Epoch 2/50
418813/418813 [==============================] - 28s 67us/step - loss: 4.9827 - acc: 0.1981
Epoch 3/50
418813/418813 [==============================] - 28s 67us/step - loss: 4.4842 - acc: 0.2476
Epoch 4/50
418813/418813 [==============================] - 28s 66us/step - loss: 4.1660 - acc: 0.2774
Epoch 5/50
418813/418813 [==============================] - 28s 66us/step - loss: 3.9094 - acc: 0.3003
Epoch 6/50
418813/418813 [==============================] - 28s 67us/step - loss: 3.7207 - acc: 0.3173
Epoch 7/50
418813/418813 [==============================] - 28s 68us/step - loss: 3.5447 - acc: 0.3338
Epoch 8/50
418813/418813 [==============================] - 28s 67us/step - loss: 3.3969 - acc: 0.3491
Epoch 9/50
418813/418813 [==============================] - 28s 67us/step - loss: 3.2609 - acc: 0.3646
Epoch 10/50
418813/418813 [==============================] - 28s 66us/ste

## Note on "temperature" to alter a distribution
A parameter called **"temperature(softmax temperature)"** is used to alter the original distribution below.
<br>
<br>
$$\exp(\frac{\log(original\ distribution)}{temperature})$$
$$= exp(\log(original\ distribution)) \times exp(-temperature)$$
$$= original\ distribution \times \exp(-temperature)$$
$$where\ 0 \neq temperature$$
$$(use\ argmax(original distribution)\ where\ temperature = 0)$$
<br>
That is, by multiplying original distribution by a number which is less than 1(equivalent to say temperature > 0), difference between high probabilities of some indexes and low probabilities of other indexes become small, so that the original distribution will have larger entropy(getting closer to uniform distribution, which is more unpredictable)

In [7]:
def alter_dist_and_pick_one(orig_dist, temp):
    if temp == 0:
        return np.argmax(orig_dist)
    else:
        dist = np.log(orig_dist + 1e-7) / temp
        dist = np.exp(dist)
        normalized_dist = dist / np.sum(dist)
        choices = range(len(normalized_dist))
        return np.random.choice(choices, p=normalized_dist)

In [8]:
def generate_text(model, max_len, temp=0.1):
    generated_text = "<【Sports Watch】日本を代表する打者である松井秀喜氏は"
    for i in range(max_len):
        int_tokens = tokenizer.EncodeAsIds(generated_text)
        #update int_tokens to predict next token
        int_tokens = int_tokens[-seq_length:]
        int_tokens = np.reshape(int_tokens, (1,seq_length))
        pred_dist = model.predict(int_tokens, verbose=0)[0]
        
        pred_index = alter_dist_and_pick_one(pred_dist, temp)
        generated_text += tokenizer.IdToPiece(int(pred_index))
        
        if generated_text[-1] == ">" or generated_text[-3:] == "▁▁▁":
            break
        
    return generated_text

In [16]:
result = generate_text(model, 1000, 0.3)
print(result)

<【Sports Watch】日本を代表する打者である松井秀喜氏は「あんま怒りだな」「ジーコも時代遅れだ」「けは何なのか」と呟き、一転、自らが来いなければ中日であり、日本選手に加入して2シーズンを無失点の登板を生めぐる。▁▁今年は全国から45歳の武田修宏コーチを退任をまとめ役。欧州王者を更新するの日本男子17年ぶりの金メダル獲得に、9日に脳障害により引退危機「妊娠したら、20歳の女子高生チャンピオン」とも称される日々を買ってしまった。▁このニュースは複数のテレビ局の報道が流れ始めたことにも関わらず、この鼻水準を(担当者が公開パがいました)な。ミーティングした点と差があると思ってたら」と語った。▁▁▁
